What I have in the first tutorial is pretty good.
Download cbmc 6

Look into floating point stuff at the same time?
Simpler?
PID
Kalman
Kepler / Baylonian

https://haslab.github.io/MFES/2122/CBMCexamples-handout.pdf



In [40]:
%%file /tmp/ex1.c
void main (void)
{
 int x;
 int y=8, z=0, w=0;
 if (x)
    z = y - 1;
 else
    w = y + 1;
 assert (z == 7 || w == 9);
}

Writing /tmp/ex1.c


In [42]:
! cbmc /tmp/ex1.c --show-vcc

CBMC version 6.0.1 (cbmc-6.0.1-5-g54c20cdb91) 64-bit x86_64 linux
Type-checking ex1
file /tmp/ex1.c line 9 function main: function 'assert' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Starting Bounded Model Checking
VERIFICATION CONDITIONS:

file /tmp/ex1.c line 9 function main
assertion z == 7 || w == 9
{-1} __CPROVER_dead_object#1 = NULL
{-2} __CPROVER_deallocated#1 = NULL
{-3} __CPROVER_max_malloc_size!0#1 = 36028797018963968
{-4} __CPROVER_memory_leak#1 = NULL
{-5} __CPROVER_rounding_mode!0#1 = 0
{-6} main::1::y!0@1#2 = 8
{-7} main::1::z!0@1#2 = 0
{-8} main::1::w!0@1#2 = 0
{-9} goto_symex::\guard#1 ⇔ ¬(main::1::x!0@1#1 = 0)
{-10} main::1::z!0@1#3 = 7
{-11} main::1::w!0@1#3 = 9
{-12} main::1::x!0@1#3 = (goto_symex::\guard#1 ? main::1::x!0@1#1 : 0)
{-13} main::1::z!0@1#4 = (goto_symex::\guard#1 ? 7 : 0)
{-14} main::1::w!0@1#4 = (goto_symex::\guard#1 ? 0 : 9)
├──────────────

In [43]:
%%file /tmp/ex2.c
void main (void)
{
 int x;
 int y=8, z=0, w=0;
 if (x)
 z = y - 1;
 else
 w = y + 1;
 assert (z == 5 || w == 9);
}


Writing /tmp/ex2.c


In [45]:
! cbmc /tmp/ex2.c --trace

CBMC version 6.0.1 (cbmc-6.0.1-5-g54c20cdb91) 64-bit x86_64 linux
Type-checking ex2
file /tmp/ex2.c line 9 function main: function 'assert' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Starting Bounded Model Checking
Passing problem to propositional reduction
converting SSA
Running propositional reduction
SAT checker: instance is SATISFIABLE
Building error trace

** Results:
/tmp/ex2.c function main
[main.overflow.1] line 6 arithmetic overflow on signed - in y - 1: SUCCESS
[main.overflow.2] line 8 arithmetic overflow on signed + in y + 1: SUCCESS
[main.assertion.1] line 9 assertion z == 5 || w == 9: FAILURE

Trace for main.assertion.1:

State 11 file /tmp/ex2.c function main line 3 thread 0
----------------------------------------------------
  x=1024 (00000000 00000000 00000100 00000000)

State 13 file /tmp/ex2.c function main line 4 thread 0
---------------------------------

https://github.com/awslabs/aws-c-common/blob/main/include/aws/common/assert.h 
This is an interesting file. They replace assume with forms of __builtin_unreachable. What does gcc do with that? Does that enhance optimizations?

```


```

In [ ]:
%%file /tmp/array_list.inl

AWS_STATIC_IMPL
int aws_array_list_init_dynamic(
    struct aws_array_list *AWS_RESTRICT list,
    struct aws_allocator *alloc,
    size_t initial_item_allocation,
    size_t item_size) {

    AWS_FATAL_PRECONDITION(list != NULL); // mostly wrapper for __CProver_precondition
    AWS_FATAL_PRECONDITION(alloc != NULL);
    AWS_FATAL_PRECONDITION(item_size > 0);

    AWS_ZERO_STRUCT(*list);

    size_t allocation_size = 0;
    if (aws_mul_size_checked(initial_item_allocation, item_size, &allocation_size)) {
        goto error;
    }

    if (allocation_size > 0) {
        list->data = aws_mem_acquire(alloc, allocation_size);
        if (!list->data) {
            goto error;
        }
#ifdef DEBUG_BUILD
        memset(list->data, AWS_ARRAY_LIST_DEBUG_FILL, allocation_size);

#endif
        list->current_size = allocation_size;
    }
    list->item_size = item_size;
    list->alloc = alloc;

    AWS_FATAL_POSTCONDITION(list->current_size == 0 || list->data);
    AWS_POSTCONDITION(aws_array_list_is_valid(list));
    return AWS_OP_SUCCESS;

error:
    AWS_POSTCONDITION(AWS_IS_ZEROED(*list));
    return AWS_OP_ERR;
}

In [ ]:
%%file /tmp/aws_array_list_init_dynamic.c
// https://github.com/awslabs/aws-c-common/blob/main/verification/cbmc/proofs/aws_array_list_init_dynamic/aws_array_list_init_dynamic_harness.c
/**
 * Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
 * SPDX-License-Identifier: Apache-2.0.
 */

#include <aws/common/array_list.h>
#include <proof_helpers/make_common_data_structures.h>

/**
 * Runtime: 6s
 */
void aws_array_list_init_dynamic_harness() {
    /* data structure */
    struct aws_array_list list; /* Precondition: list is non-null */

    /* parameters */
    struct aws_allocator *allocator = aws_default_allocator(); /* Precondition: allocator is non-null */
    size_t item_size;
    size_t initial_item_allocation;

    /* assumptions */
    __CPROVER_assume(initial_item_allocation <= MAX_INITIAL_ITEM_ALLOCATION);
    __CPROVER_assume(item_size > 0 && item_size <= MAX_ITEM_SIZE);

    /* perform operation under verification */
    if (aws_array_list_init_dynamic(&list, allocator, initial_item_allocation, item_size) == AWS_OP_SUCCESS) {
        /* assertions */
        assert(aws_array_list_is_valid(&list));
        assert(list.alloc == allocator);
        assert(list.item_size == item_size);
        assert(list.length == 0);
        assert(list.current_size == item_size * initial_item_allocation);
    } else {
        /*assertions */
        assert(AWS_IS_ZEROED(list));
    }
}


In [38]:
%%file /tmp/float1.c

float calc1(float a, float b) {
    return a*a + b; // originally 2*a + b. Could try pi or something
}

double calc2(double a, double b) {
    return a*a + b;
}

int main(){
    float a,b;
    __CPROVER_assume( 1 < a && a < 2 && 1 < b && b < 2);
    double a1 = a;
    double b1 = b;
    float res1 = calc1(a,b);
    //float res2 = calc1(a,b); 
    float res2 = calc2(a1,b1);
    assert(res1 >= res2 - 1e-2 && res1 <= res2 + 1e-2); 
    //assert(res2 == res1);
    return 0;
}

Overwriting /tmp/float1.c


In [39]:
! cbmc /tmp/float1.c --trace

CBMC version 6.0.1 (cbmc-6.0.1-5-g54c20cdb91) 64-bit x86_64 linux
Type-checking float1
file /tmp/float1.c line 18 function main: function 'assert' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Starting Bounded Model Checking
Passing problem to propositional reduction
converting SSA
Running propositional reduction
SAT checker: instance is UNSATISFIABLE

** Results:
/tmp/float1.c function main
[main.assertion.1] line 18 assertion (double)res1 >= (double)res2 - 0.01 && (double)res1 <= (double)res2 + 0.01: SUCCESS

** 0 of 1 failed (1 iterations)
VERIFICATION SUCCESSFUL


In [ ]:
%%file /tmp/vec2.c

#include <assert.h>
#include <math.h>
float dot_f(float[2] a, float[2]b) {
    return a[0]*b[0] + a[1]*b[1];
}

double dot_d(double[2] a, double[2]b) {
    return a[0]*b[0] + a[1]*b[1];
}

int main(){
    float a[2],b[2];
    double a1[2] = {a[0], a[1]};
    double b1[2] = {b[0], b[1]};
    float res1 = dot_f(a,b);
    double res2 = dot_d(a1,b1);
    assert(res1 >= res2 - 1e-6 && res1 <= res2 + 1e-6); 
    return 0;
}

cross product.
abs
add
rotation
quaternion

knuckledragger?

halide?
Something image?